In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2394,2024-06-22,Austrália Nbl1 Oeste,06:00,Warwick Senators F,South West Slammers F,1.77,1.92,160.5,1.85,1.85,0.0,0.00,0.00,hMH8A8MC,0.564972,0.520833,0.540541,0.540541,0.085805,157.020,125.533701,0.799476,3.0,0.000000,0.000000,95.48,1.636,1.195776,0.730914,79.0,1109.792,614.676816,0.553867,0.0,0.000000,NaN,633.60,19.378,8.922747,0.460458,-177.0,77,48,1.24,13.20,111.692,961.146,0.057488,0.000000,NaN,-0.40,-0.080,-9.625000,0.000000,0.0,0.000000,-5.00,-1.000,-0.920000,0.000000,0.0,0.000000
2395,2024-06-22,Chile Lnb 2,21:00,Arturo Prat San Felipe,Aleman,5.40,1.13,150.5,1.85,1.85,0.0,0.00,0.00,rZIOhtir,0.185185,0.884956,0.540541,0.540541,0.070141,615.388,549.898515,0.893580,1.2,1.643168,1.369306,467.60,9.294,10.009529,1.076988,-62.0,132.146,27.818106,0.210510,1.8,1.643168,0.912871,131.67,1.758,0.426579,0.242650,27.0,70,77,6.68,1.71,385.504,180.630,0.924761,0.000000,NaN,-1.39,-0.278,-15.827338,0.000000,0.0,0.000000,-1.38,-0.276,-0.471014,0.000000,0.0,0.000000
2396,2024-06-22,Chile Lnb 2,19:30,Quilpue Basquetbol,Colo Colo,11.70,1.01,144.5,1.85,1.85,0.0,0.00,0.00,CjShb2TK,0.085470,0.990099,0.540541,0.540541,0.075569,113.116,51.006140,0.450919,1.2,1.643168,1.369306,197.34,1.670,0.619879,0.371185,-40.0,109.472,15.871658,0.144984,3.0,0.000000,0.000000,117.30,1.186,0.192042,0.161924,130.0,78,115,2.53,1.02,0.000,0.000,1.189453,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
2397,2024-06-22,Chile Lnb 2,19:00,Truenos De Talca,Stadio Italiano,1.38,2.88,135.5,1.85,1.85,0.0,0.00,0.00,MPHSi06l,0.724638,0.347222,0.540541,0.540541,0.071860,157.398,60.751054,0.385971,1.8,1.643168,0.912871,138.04,2.340,1.109775,0.474263,35.0,215.186,211.214700,0.981545,1.2,1.643168,1.369306,86.42,3.226,3.356878,1.040570,9.0,68,58,2.03,1.49,141.974,206.806,0.497963,0.000000,NaN,0.42,0.084,4.523810,0.000000,0.0,0.000000,-2.34,-0.468,-4.017094,0.000000,0.0,0.000000
2398,2024-06-22,Chile Lnb 2,17:00,Boston C. C.,Arabe Valparaiso,1.27,3.53,137.5,1.85,1.85,0.0,0.00,0.00,jsRdcMrR,0.787402,0.283286,0.540541,0.540541,0.070688,193.884,157.105726,0.810308,2.4,1.341641,0.559017,143.62,2.916,3.066534,1.051623,37.0,125.186,58.552031,0.467720,1.2,1.643168,1.369306,62.23,1.964,0.909742,0.463209,-35.0,86,49,1.67,1.27,143.876,148.180,0.665859,0.000000,NaN,0.73,0.146,1.849315,0.000000,0.0,0.000000,0.81,0.162,15.617284,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,2024-06-22,Austrália Nbl1 Sul,06:00,Bendigo Braves,Keilor Thunder,1.64,2.18,161.5,1.84,1.84,-2.5,1.85,1.94,EX14Lf13,0.609756,0.458716,0.543478,0.543478,0.068472,215.502,86.500779,0.401392,1.2,1.643168,1.369306,240.48,2.922,1.513347,0.517915,-20.0,134.308,26.716727,0.198921,2.4,1.341641,0.559017,127.82,1.792,0.350243,0.195448,23.0,72,77,3.34,1.66,252.006,252.112,0.199915,0.000000,0.033583,2.93,0.586,1.092150,0.000000,0.0,0.000000,7.03,1.406,0.839260,0.000000,0.0,0.000000
2483,2024-06-22,Austrália Nbl1 Norte,05:30,Darwin Salties F,North Gold Coast Seahawks F,2.56,1.47,166.5,1.83,1.87,0.0,0.00,0.00,QJVooziL,0.390625,0.680272,0.546448,0.534759,0.070897,20

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
2394,06:00,Austrália Nbl1 Oeste,Warwick Senators F,South West Slammers F,1.77,Back Home
2461,02:00,Austrália Nbl1 Leste,Manly W. F,Newcastle Falcons F,1.62,Back Home
2481,06:00,Austrália Nbl1 Sul,Geelong,Hobart Chargers,1.64,Back Home
